In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import nn
import numpy as np

import matplotlib.pyplot as plt

In [20]:
# Check available device
if torch.cuda.is_available():
    print('cuda')
    device = 'cuda'
elif torch.backends.mps.is_available():
    print('mps')
    device ='mps'
else:
    print('cpu')
    device ='cpu'

mps


# PyTorch Tensor

In [30]:
# Create tensor
# torch.tensor(data, *, dtype=None, device=None, requires_grad=False, pin_memory=False) → Tensor


# from list
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

# from np array
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

# from shape
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(x_data)
print(x_np)
print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])
Random Tensor: 
 tensor([[0.9140, 0.9597, 0.3849],
        [0.8009, 0.8925, 0.9602]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [3]:
# Tensor attribute

tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

if torch.backends.mps.is_available():
    tensor = tensor.to("mps")
    print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu
Device tensor is stored on: mps:0


In [25]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

In [12]:
# Tensor operations
# https://pytorch.org/docs/stable/torch.html#math-operations

# from shape
shape = (2,2)

t1 = torch.full(shape, 3.0, device='mps')
data = [[1.0, 2],[3, 4]]
t2 = torch.tensor(data, device='mps')

print('ADD:', t1+t2)
# MPS doesn't support int op(?) -> better to make it float32
print("Matrix mul:", torch.matmul(t1, t2))
print('Sum:', t1.sum())

# In-place op
print('In-place Add:', t1.add_(3.))

ADD: tensor([[4., 5.],
        [6., 7.]], device='mps:0')
Matrix mul: tensor([[12., 18.],
        [12., 18.]], device='mps:0')
Sum: tensor(12., device='mps:0')
In-place Add: tensor([[6., 6.],
        [6., 6.]], device='mps:0')


---

# Dataset & Loader

In [13]:
# Load fashion_mnist dataset

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [10]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [18]:
# DataLoader can be used as an iterator with the whole set,
# Each iter is one batch
train_dataloader = DataLoader(training_data, batch_size=3000, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

print(len(train_dataloader.dataset), len(test_dataloader.dataset))

60000 10000


In [25]:
j=0
for i in train_dataloader:
    # sample, label
    print(i[0].shape, i[1].shape)
    break

torch.Size([3000, 1, 28, 28]) torch.Size([3000])


In [21]:
class FirstModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, input):
        x = self.flatten(input)
        logits = self.linear_relu_stack(x)
        return logits

model = FirstModel().to(device)
print(model)

FirstModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [25]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Training loop
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Reset grad
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.285364  [ 3000/60000]
Test Error: 
 Accuracy: 20.1%, Avg loss: 2.281720 

Epoch 2
-------------------------------
loss: 2.281603  [ 3000/60000]
Test Error: 
 Accuracy: 21.4%, Avg loss: 2.278703 

Epoch 3
-------------------------------
loss: 2.278545  [ 3000/60000]
Test Error: 
 Accuracy: 23.2%, Avg loss: 2.275679 

Epoch 4
-------------------------------
loss: 2.276697  [ 3000/60000]
Test Error: 
 Accuracy: 24.9%, Avg loss: 2.272869 

Epoch 5
-------------------------------
loss: 2.273180  [ 3000/60000]
Test Error: 
 Accuracy: 26.6%, Avg loss: 2.269861 

Done!
